In [123]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import os

os.chdir("/content/drive/MyDrive/Colab Notebooks/")

In [125]:
comments_df = pd.DataFrame(columns = ["ID", "comment"])

for page in range(1, 43):
    print(page)
    url = "https://www.fmkorea.com/index.php?document_srl=5096598408&mid=humor&cpage=" + str(page) + "#5096598408_comment"
    
    html = urlopen(url)
    bsObject = BeautifulSoup(html, "html.parser")

    ## data preprocessing ##
    comment = bsObject.find("ul", {"class":"fdb_lst_ul"})
    comments = comment.get_text().splitlines()
    cc = [c for c in comments]                             # 나뉜 string들 합치기
    dd = "".join(cc).strip()                               # trailing whitespace 제거
    ee = re.sub(r"\t\d?", "      ", dd)                    #\t\t\t3\t 등 제거
    ee2 = re.sub(r"\d+\.\d+\.\d+\s\d+\:\d+", "______", ee) # 댓글 날짜 제거
    ee3 = re.sub(r"\d+\s시간\s전", "______", ee2)            # 댓글 시점 제거
    ff = [i.strip() for i in ee3.split("      ") if len(i.strip()) > 2] # 리스트로 쪼개고 너무 짧은 element 제거, 배뎃제거
    ###

    current_n_rows = comments_df.shape[0]
    
    for i in range(0, len(ff)):
        temp_line = ff[i].split("______")
        
        try:
            id = temp_line[0].strip()
            c = temp_line[1].strip()
            comments_df.loc[len(comments_df)] = [id, c]
        except: 
            pass
       
comments_df.to_csv("comments_raw.csv", index = False)

URLError: ignored

In [ ]:
# comments_df.to_csv("comments_raw.csv")
## 제외:
### [삭제된 댓글입니다.]
### 중복 ID

deleted_idx = []
for c in range(0, len(comments_df)):
    full_string = comments_df.loc[c, "comment"]
    if "[삭제된 댓글입니다.]" in full_string:
        deleted_idx.append(c)
# print(deleted_idx)
comments_df2 = comments_df.drop(deleted_idx, axis = 0).drop_duplicates(subset = ["ID"], keep = "first").reset_index(drop = True)
comments_df2.to_csv("comments_remove_dups.csv", index = False)
comments_df2


,ID,comment
0,구름위로,그놈의 vs질 참 지겹다왕석현100명vs뱅갈호랑이1마리
1,민나상고멘,구름위로 이건 석현이 의견도 들어봐야..
2,괴해신살권,구름위로 김성찬150명 vs 왕석현100명
3,비정상인,2000년생 이상 급식은 장범준90년대생이하는 성시경 나올듯
4,래드키,비정상인 반대임ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ00년생은 성시경 80년생은 장범준인 분위기였음ㅋㅋ...
...,...,...
980,내가하면로맨스,벚꽃엔딩이 사기긴해
981,A.Gordon,댓글 존나 많네 시박 ㅋㅋ
982,닭고기스프,ㅋㅋ별걸 다 가지고 싸우네 개취가지고난 저음 노래를 너무 싫어해서 장범준 노래 좋은...
983,C.Parker,그만해 ㅋㅋㅋ


In [120]:
comments_df = pd.read_csv("comments_raw.csv")
comments_df2 = pd.read_csv("comment_remove_dups.csv")

## split 성시경, 장범준 dfs

sung_keywords = ["시경", "성식", "거리에서", "두사람"]
jang_keywords = ["범준", "벚꽃", "여수", "노래방에서"]

sung_df = pd.DataFrame(columns = ["ID", "comments"])
jang_df = pd.DataFrame(columns = ["ID", "comments"])

for idx_c in range(0, len(comments_df2)):
    
    temp_id, temp_comment = comments_df2.iloc[idx_c]
    
    add_s = 0
    add_j = 0
    
    for sk in sung_keywords:
        
        for jk in jang_keywords:
        
            if sk in temp_comment and jk not in temp_comment:
                add_s = add_s + 1
            elif jk in temp_comment and sk not in temp_comment:
                add_j = add_j + 1
            
    if add_s >= 1:
        sung_df.loc[len(sung_df)] = [temp_id, temp_comment]

    if add_j >= 1:
        jang_df.loc[len(jang_df)] = [temp_id, temp_comment]

sung_df.to_csv("comments_remdup_sung.csv", index = False)
jang_df.to_csv("comments_remdup_jang.csv", index = False)

ValueError: ignored

In [ ]:
sung_df

,ID,comments
0,비정상인,2000년생 이상 급식은 장범준90년대생이하는 성시경 나올듯
1,래드키,비정상인 반대임ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ00년생은 성시경 80년생은 장범준인 분위기였음ㅋㅋ...
2,BEST 래드키,비정상인 성시경 전성기를 모르니까 풍문으로 들은 애들이 더 고평가엠팍은 나이대가 높...
3,델카이저,래드키 이말이맞음 성시경이 00년대 goat 은 아니자너
4,원베일리,"델카이저 근데 장범준이 20년대 goat도 아니잖아 ;가창력은 성시경이 우위고,히트..."
...,...,...
444,radfkjas,성시경하고 장범준하고 가창력 차이가 너무 넘사고 장범준이 벚꽃연금이 있는데데뷔년도만...
445,월월콧콧,성시경 저평가?ㅋㅋ 전 글 베댓 긁어와드려?
446,해냈구나구루자,성시경 이윽고 들으면 와시발 소리나오지만장범준 벚꽃엔딩 흔꽃샴 들어도 와시발소리까진...
447,최시은,노알못성시경 몇몇 노래 완전 듣기 좋음장범준 많은 노래 듣기 좋음


In [ ]:
jang_df

,ID,comments
0,비정상인,2000년생 이상 급식은 장범준90년대생이하는 성시경 나올듯
1,래드키,비정상인 반대임ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ00년생은 성시경 80년생은 장범준인 분위기였음ㅋㅋ...
2,BEST 래드키,비정상인 성시경 전성기를 모르니까 풍문으로 들은 애들이 더 고평가엠팍은 나이대가 높...
3,원베일리,"델카이저 근데 장범준이 20년대 goat도 아니잖아 ;가창력은 성시경이 우위고,히트..."
4,난그런거몰라,원베일리 상업성 고려하면 장범준이 맞겠네.
...,...,...
444,EIIE,가창력은 성이 위지만 후대에 기억되는건 장범준이라는건 공감불가 장범준이 우리가 가지...
445,호사카Yuji,장범준 노래 메들리로 연결해서 들으면 언제 노래가 바뀐지 알아차릴 수 없고 제목 못...
446,내가하면로맨스,벚꽃엔딩이 사기긴해
447,닭고기스프,ㅋㅋ별걸 다 가지고 싸우네 개취가지고난 저음 노래를 너무 싫어해서 장범준 노래 좋은...


In [92]:
comments_df = pd.read_csv("comments_raw.csv")
comments_df2 = pd.read_csv("comments_remove_dups.csv")
comments_sung = pd.read_csv("comments_remdup_sung.csv")
comments_jang = pd.read_csv("comments_remdup_jang.csv")

In [ ]:
## sentiment analysis using CLOVA sentiment

import sys
import requests
import json

client_id = "dgf7iav5gn"
client_secret = "1pTSu6qoFJ1tctVGLRA5oPXUo8j39GbALRFcAYhF"

url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

### Merge the comments into one document.
### Sentences are separated by "."
result_df = pd.DataFrame(columns = ["content", "sentiment", "negative", "neutral", "positive", "highlight"])

n = 0
m = 999

document = ".".join(comments_sung.comments)
document = document + "."
max_doc_length = len(document)

while n < max_doc_length:
    
    m = m if m <= max_doc_length else max_doc_length
    
    content = document[n:m]
    while content[-1] != ".":
        content = content[0:len(content)-1]

    ### Run sentiment analysis
    data = {
        "content": content
    }

    # print(json.dumps(data, indent = 4, sort_keys = True))
    response = requests.post(url, data = json.dumps(data), headers = headers)
    rescode = response.status_code
    if (rescode == 200):
        print(response.text)
    else:
        print("Error : " + response.text)

    text = response.json()

    for i in range(0, len(text['sentences'])):
        c = text["sentences"][i]["content"]
        sent = text["sentences"][i]["sentiment"]
        sent_neg = text["sentences"][i]['confidence']['negative']
        sent_neu = text["sentences"][i]['confidence']['neutral']
        sent_pos = text["sentences"][i]['confidence']['positive']
        offset = text["sentences"][i]['highlights'][0]["offset"]
        length = text["sentences"][i]['highlights'][0]["length"]
        highlight = text['sentences'][i]["content"][offset: offset + length]

        result_df.loc[len(result_df), ] = [c, sent, sent_neg, sent_neu, sent_pos, highlight]

    n = n + len(content)
    m = n + 999 if n + 999 <= max_doc_length else max_doc_length

result_df.to_csv("sentiment_result_sung.csv")
result_df

In [94]:
sung_result = pd.read_csv("sentiment_result_sung.csv")
jang_result = pd.read_csv("sentiment_result_jang.csv")

In [101]:
sung_result.loc[sung_result['sentiment'] == "positive", ]

,Unnamed: 0,content,sentiment,negative,neutral,positive,highlight
6,6,"가창력은 성시경이 우위고,히트곡은 장범준이 우위고..근데 또 20년간 술담배 다 하...",positive,0.000435,0.012596,0.986969,그 많은 임팩트있는 곡을 썼다는걸 생각하면 또 장범준
8,8,래드키 나이대 좀 있는 사람들한테 성시경은 음악성이나 가창력보다는 여자들한테 유독 ...,positive,0.000436,0.005999,0.993565,있는 사람들한테 성시경은 음악성이나 가창력보다는 여자들한테 유독 인기 많은 가수 정...
10,10,그리고 오히려 장범준이 약간 독보적인 부분이 있어서 나름 고평가 가능함.,positive,0.000624,0.002318,0.997059,부분이 있어서 나름 고평가 가능함
13,13,"이승철, 성시경 쪽이 뭔가 아티스트, 대가 이미지가 있는데 정작 작사 작곡은 임창...",positive,0.001151,0.101942,0.896907,차트 순위 같은 기록도 더 좋음
14,14,성시경은 거기에 끼는 사람인거고장범준은 포크와 싱어송라이터 부문에서 독보적임,positive,0.000469,0.005930,0.993601,성시경은 거기에 끼는 사람인거고장범준은 포크와 싱어송라이터 부문에서 독보적임
...,...,...,...,...,...,...,...
675,675,기량을 떠나서 장범준이 킬링보이스 나오면 성시경의 킬링보이스 조회수 3800만 명은...,positive,0.000917,0.000703,0.998380,성시경의 킬링보이스 조회수 3800만 명은 못찍을거같음
676,676,난 전반적으로 성시경노래가 더좋음.,positive,0.001077,0.000309,0.998615,난 전반적으로 성시경노래가 더좋음
691,691,중학교시절 잭스키스 hot멤버 숫자도 모를만큼티비도 안보고 알못인데 나이가 많아...,positive,0.001679,0.000238,0.998082,나이가 많아 그런가성시경 미소천사 희재 넌 감동이었어
696,696,노알못성시경 몇몇 노래 완전 듣기 좋음장범준 많은 노래 듣기 좋음.,positive,0.001139,0.000291,0.998570,좋음장범준 많은 노래 듣기 좋음


In [100]:
jang_result.loc[jang_result['sentiment']=="positive", ]

,Unnamed: 0,content,sentiment,negative,neutral,positive,highlight
5,5,"가창력은 성시경이 우위고,히트곡은 장범준이 우위고..근데 또 20년간 술담배 다 하...",positive,0.000920,0.107344,0.891736,원베일리 상업성 고려하면 장범준이 맞겠네
7,7,벚꽃엔딩은 10년대 대한민국에서 가장 히트곡이라고 할 수있는데.래드키 나이대 좀 있...,positive,0.000556,0.017599,0.981846,있는 사람들한테 성시경은 음악성이나 가창력보다는 여자들한테 유독 인기 많은 가수 정...
9,9,그리고 오히려 장범준이 약간 독보적인 부분이 있어서 나름 고평가 가능함.,positive,0.000624,0.002318,0.997059,부분이 있어서 나름 고평가 가능함
12,12,"이승철, 성시경 쪽이 뭔가 아티스트, 대가 이미지가 있는데 정작 작사 작곡은 임창...",positive,0.001151,0.101942,0.896907,차트 순위 같은 기록도 더 좋음
13,13,성시경은 거기에 끼는 사람인거고장범준은 포크와 싱어송라이터 부문에서 독보적임,positive,0.000469,0.005930,0.993601,성시경은 거기에 끼는 사람인거고장범준은 포크와 싱어송라이터 부문에서 독보적임
...,...,...,...,...,...,...,...
682,682,기량을 떠나서 장범준이 킬링보이스 나오면 성시경의 킬링보이스 조회수 3800만 명은...,positive,0.000917,0.000703,0.998380,성시경의 킬링보이스 조회수 3800만 명은 못찍을거같음
690,690,흔들리는 꽃들 속에서 네샴푸향 다 너무 좋음.,positive,0.000170,0.000083,0.999747,흔들리는 꽃들 속에서 네샴푸향 다 너무 좋음
698,698,중학교시절 잭스키스 hot멤버 숫자도 모를만큼티비도 안보고 알못인데 나이가 많아...,positive,0.001679,0.000238,0.998082,나이가 많아 그런가성시경 미소천사 희재 넌 감동이었어
701,701,노알못성시경 몇몇 노래 완전 듣기 좋음장범준 많은 노래 듣기 좋음.,positive,0.001139,0.000291,0.998570,좋음장범준 많은 노래 듣기 좋음


In [95]:
print(sum(sung_result['negative']))
print(sum(sung_result['neutral']))
print(sum(sung_result['positive']))

160.4383892923194
376.4898553432385
162.07175638054153


In [96]:
print(sum(jang_result['negative']))
print(sum(jang_result['neutral']))
print(sum(jang_result['positive']))

169.15001738341704
365.89908997673626
173.95089290541642
